In [1]:
# %load_ext autoreload # uncomment this line if you want to reload modules automatically
# %autoreload 2 # uncomment this line if you want to reload modules automatically

import torch
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from ipywidgets import interact

from scripts.imagedata import ImageData
from scripts.neutralliner import NeutralLiner
from scripts.config import device, PATH_TO_MCINTOSH

# No pretrain

In [ ]:
filenames = os.listdir(PATH_TO_MCINTOSH)[:28]

num_epochs = 10000
arch = [3, 6, 12, 24, 12, 6, 3, 1]
lr = 1e-3
weight_decay = 1e-4

for filename in filenames:
    model = NeutralLiner(image_list=[ImageData(path_or_img=os.path.join(PATH_TO_MCINTOSH, filename), data_mode='fits')],
                lr=lr, arch=arch, mode='3d', weight_decay=weight_decay)
    model.to(device)
    model.start_training(num_epochs=num_epochs, need_plot=True, need_save=False)

# Semi-supervised learning

In [4]:
model = NeutralLiner(image_list=[ImageData(os.path.join(PATH_TO_MCINTOSH, os.listdir(PATH_TO_MCINTOSH)[0]), data_mode='fits')],
                     lr=5e-3,
                     mode='3d',
                     arch=[3, 6, 12, 24, 12, 6, 3, 1],
                     weight_decay=1e-4)
model.to(device)
model.start_training(num_epochs=2000, need_plot=False)

In [7]:
model.image_list[0].height

512

In [ ]:
model.test_model()

0